### Tutorial on Space-time FEM with FEniCSx
Dominik Kern ORCID [0000-0002-1958-2982](https://orcid.org/0000-0002-1958-2982) 

This notebook is a supplement to the tutorial with doi [10.5281/zenodo.16761462](https://zenodo.org/records/16761462)

**Solving the non-dimensional wave equation in a 1D bar using time-stepping with the leap-frog method**

In [ ]:
import dolfinx
from dolfinx import geometry
from dolfinx.fem import Function, form
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import create_unit_interval
from dolfinx.io import VTXWriter
from ufl import TrialFunction, TestFunction, dx, inner, grad, sin, pi, SpatialCoordinate
from petsc4py import PETSc
from mpi4py import MPI
import numpy as np
import pyvista as pv

#### parameters

In [ ]:
nx = 4  # Number of spatial cells
nt = 8 # observation, may not be less the 2*nx*order
order = 1 # Polynomial order for spatial FEM only (time-stepping is fixed)
T = 1.0  # Total time (unit interval)
dt = T/nt  # Time step 

#### discretization

In [ ]:
domain = create_unit_interval(MPI.COMM_WORLD, nx)   
V = dolfinx.fem.functionspace(domain, ("Lagrange", order))

u = TrialFunction(V)
Du = TestFunction(V)

# 3. Time-stepping variables
u_n = Function(V)  # u at current time step (n)
u_n_minus_1 = Function(V)  # u at previous time step (n-1)
u_n_plus_1 = Function(V)  # u at next time step (n+1)

# 4. Initial conditions
u_n.interpolate(lambda x: np.sin(np.pi * x[0]))
u_n_minus_1.interpolate(lambda x: np.sin(np.pi * x[0])) # Assuming zero initial velocity, i.e. same displacements on the time before start

# 5. Weak form for Leap-Frog method
a = inner(u, Du) * dx  # This will be the mass matrix
L = (2 * u_n - u_n_minus_1) * Du * dx - dt**2  * inner(grad(u_n), grad(Du)) * dx

# 6. Boundary Conditions (Homogeneous Dirichlet at x=0 and x=Lx)
left_boundary = lambda x: np.isclose(x[0], 0.0)
right_boundary = lambda x: np.isclose(x[0], 1.0)

left_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, left_boundary)
right_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, right_boundary)

left_dofs = dolfinx.fem.locate_dofs_topological(V, domain.topology.dim - 1, left_facets)
right_dofs = dolfinx.fem.locate_dofs_topological(V, domain.topology.dim - 1, right_facets)

bcs = [dolfinx.fem.dirichletbc(PETSc.ScalarType(0.0), left_dofs, V),
    dolfinx.fem.dirichletbc(PETSc.ScalarType(0.0), right_dofs, V)]

##### solution

In [ ]:
problem = LinearProblem(a, L, bcs=bcs, u=u_n_plus_1, petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})

# sort result by x-coordinate to synchronize with plot-grid
x_coords = V.tabulate_dof_coordinates()[:, 0]
sort_order = np.argsort(x_coords)
u_sol = np.zeros((nt+1, order*nx+1))
u_sol[0, :] = u_n.x.array[sort_order]

t = 0.0
for n in range(nt):
    t += dt
    problem.solve()

    # Update for next time step
    u_n_minus_1.x.array[:] = u_n.x.array
    u_n.x.array[:] = u_n_plus_1.x.array

    x_coords = V.tabulate_dof_coordinates()[:, 0]
    sort_order = np.argsort(x_coords)
    u_values = u_n.x.array
    u_sol[n+1, :] =  u_values[sort_order]

#### post-processing

In [ ]:
xt = np.meshgrid(np.linspace(0, 1, order*nx+1), np.linspace(0, T, nt+1), indexing='ij')
X, T = xt  

u_grid = u_sol.T  

points = np.zeros((X.size, 3))
points[:, 0] = X.ravel(order="F")  # x
points[:, 1] = T.ravel(order="F")  # t
points[:, 2] = u_grid.ravel(order="F")  # u as height

grid = pv.StructuredGrid()
grid.points = points
grid.dimensions = [X.shape[0], X.shape[1], 1]
grid["u"] = u_grid.ravel(order="F")

plotter = pv.Plotter()
plotter.add_mesh(grid, scalars="u", cmap="viridis", show_edges=True, scalar_bar_args={'vertical':True})
plotter.show_grid(xlabel="x", ylabel="t", zlabel="u")
plotter.show()